# 🎯 LangGraph Command Object

## What is `Command`?

`Command` is a powerful control flow object in LangGraph that allows nodes to **explicitly specify the next node to execute** and **update the state** in a single return statement.

## When to Use `Command`

Use `Command` when you need:

1. **Dynamic Routing** - Decide the next node at runtime based on state or conditions
2. **Explicit Control Flow** - Override default edge behavior with programmatic navigation
3. **State Updates + Navigation** - Update state and specify destination in one operation
4. **Human-in-the-Loop Patterns** - Conditionally route to approval/rejection nodes
5. **Complex Workflows** - Replace multiple `add_edge()` calls with dynamic routing logic

## Why Use `Command`?

### ✅ Advantages:
- **Flexible**: Change routing logic without modifying graph structure
- **Cleaner Code**: Combines state update + navigation in one return
- **Dynamic**: Route based on runtime conditions (API responses, user input, etc.)
- **Readable**: Intent is clear from the node function itself

### ⚠️ When NOT to Use:
- Simple linear workflows (use `add_edge()` instead)
- Fixed routing that never changes (use conditional edges)
- When graph structure should be visually clear upfront

## Command Structure

```python
return Command(
    goto="next_node_name",  # or END to finish
    update={"key": "value"}  # State updates (optional)
)
```

## Example Use Cases

1. **Approval Workflow**: Route to "approve" or "reject" nodes based on conditions
2. **Multi-step Processing**: Chain nodes dynamically (A → B → C)
3. **Error Handling**: Route to error handler or retry nodes
4. **Conditional Completion**: Go to END early or continue processing

---


In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import TypedDict

class State(TypedDict):
    text: str

# -----------------------------
# 📌 Node A - First Step
# -----------------------------
# Uses Command to:
# 1. Explicitly route to "node_b" (dynamic routing)
# 2. Update state by appending "a" to text
# 
# WHY Command? Instead of using add_edge("node_a", "node_b"),
# we return Command for programmatic control over the next node
def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b",  # Explicitly navigate to node_b
        update={
            "text": state["text"] + "a"  # Append 'a' to current text
        }
    )

# -----------------------------
# 📌 Node B - Second Step
# -----------------------------
# Uses Command to:
# 1. Continue the chain to "node_c"
# 2. Update state by appending "b"
#
# WHY Command? Demonstrates chaining multiple nodes dynamically
# without defining edges in graph structure
def node_b(state: State): 
    print("Node B")
    return Command(
        goto="node_c",  # Explicitly navigate to node_c
        update={
            "text": state["text"] + "b"  # Append 'b' to current text
        }
    )

# -----------------------------
# 📌 Node C - Final Step
# -----------------------------
# Uses Command to:
# 1. Signal completion with goto=END
# 2. Update state with final value
#
# WHY Command? Allows node to both update state AND terminate
# the workflow programmatically
def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END,  # Explicitly end the workflow
        update={
            "text": state["text"] + "c"  # Append 'c' to current text
        }
    )

# -----------------------------
# 🏗️ Build Graph
# -----------------------------
graph = StateGraph(State)

# Add nodes to graph
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)

# Set entry point
graph.set_entry_point("node_a")

# ⚠️ NOTICE: No add_edge() calls needed!
# The Command objects in each node handle routing dynamically

# Compile the graph
app = graph.compile()

# -----------------------------
# ▶️ Execute Workflow
# -----------------------------
# Start with empty text, nodes will append "a" → "b" → "c"
response = app.invoke({
    "text": ""
})

# Expected output: {"text": "abc"}
response


Node A
Node B
Node C


{'text': 'abc'}

---

## 🔍 Key Observations

### What This Example Demonstrates:

1. **No Edge Definitions**: The graph has NO `add_edge()` calls - all routing is done via `Command`
2. **Sequential Chaining**: node_a → node_b → node_c → END, all controlled programmatically
3. **State Building**: Each node appends a letter, building "abc" progressively
4. **Dynamic Control**: Each node decides its successor at runtime (even though it's fixed here)

### Command vs. Traditional Edges

| Aspect | Traditional `add_edge()` | Using `Command` |
|--------|-------------------------|-----------------|
| **Routing** | Fixed at graph definition | Dynamic at runtime |
| **Flexibility** | Static structure | Can change based on state/conditions |
| **Use Case** | Simple linear flows | Complex conditional flows |
| **Code Location** | Graph building phase | Inside node functions |
| **Visibility** | Clear from graph structure | Hidden in node logic |

### Real-World Example: Approval Workflow

```python
def approval_node(state):
    if state["score"] > 0.8:
        return Command(goto="auto_approve", update={"status": "approved"})
    elif state["score"] > 0.5:
        return Command(goto="human_review", update={"status": "pending"})
    else:
        return Command(goto="auto_reject", update={"status": "rejected"})
```

This pattern is **impossible with static edges** - you'd need conditional edges with separate logic functions. `Command` keeps routing logic **inside the node** where it belongs.

---

## 🎓 Summary

**Use `Command` when:**
- ✅ Routing depends on runtime conditions
- ✅ You need flexible, dynamic workflows
- ✅ Human-in-the-loop or approval patterns
- ✅ You want to combine state update + navigation

**Use `add_edge()` when:**
- ✅ Simple, fixed routing
- ✅ Graph structure should be visually clear
- ✅ No conditional logic needed